# 循环神经网络


## 读取数据集


In [3]:
import torch
import random
import zipfile

with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [4]:
corpus_chars = corpus_chars.replace("\n"," ").replace("\r"," ")
corpus_chars = corpus_chars[0:10000]

## 建立字符索引

将每个字符映射成一个从0开始的连续整数，又称`索引`，来方便之后的数据处理。为了得到索引，将数据集里所有不同字符取出来，然后将其逐一映射到索引来构造词典。接着，打印`vocab_size`，即词典中不同字符的个数，又称`词典大小`。

In [5]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char,i) for i , char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1027

In [6]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print("chars:","".join([idx_to_char[idx] for idx in sample]))
print("indices:",sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [197, 239, 551, 814, 850, 900, 86, 197, 239, 707, 546, 439, 472, 58, 297, 875, 86, 197, 239, 707]


## 时序数据的采样

在训练中我们需要每次随机读取`小批量样本和标签`。`时序数据的一个样本通常包含连续的字符`。

假设时间步数为5，样本序列为5个字符，即`“想”“要”“有”“直”“升”`。

该样本的标签序列为这些字符分别在训练集中的下一个字符，即`“要”“有”“直”“升”“机”`。

有两种方式对时序数据进行采样，分别是`随机采样`和`相邻采样`。

## 随机采样

每次从数据里随机采样一个小批量。其中批量大小`batch_size`指每个小批量的样本数，`num_steps`为每个样本所包含的时间步数。 

在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机小批量`在原始序列上的位置不一定相毗邻`。

因此，无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，`每次随机采样前都需要重新初始化隐藏状态`。

In [ ]:
def data_iter_random(corpus_indices,batch_size,num_steps,device = None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) - 1)//num_steps
    epoch_size = num_examples//batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    # 返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    if device == None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for i in range(epoch_size):
        # 每次读取batch_size个样本
        i = i*batch_size
        batch_indices = example_indices[i:i+batch_size]